In [45]:
import numpy as np
import os

In [46]:
def load_data(folder_path):
    x_train = np.load(os.path.join(folder_path, 'x_train.npy'))
    y_train = np.load(os.path.join(folder_path, 'y_train.npy'))
    x_test = np.load(os.path.join(folder_path, 'x_test.npy'))
    y_test = np.load(os.path.join(folder_path, 'y_test.npy'))
    return x_train, y_train, x_test, y_test

In [47]:
x_train, y_train, x_test, y_test = load_data('lr4_dataset/')

В данной лабораторной работе будет практиковаться поиск гиперпараметров. Буду рассмотрены алгоритмы поиска гиперпараметров: grid search, random search.

Помимо поиска гиперпараметров будет рассмотрен алгоритм кросс-валидации, позволяющий получить более достоверную оценку качества модели в условиях недостатка данных.
Хотя в работе предоставлена тестовая выборка, здесь она имеет сугубо теоретический характер (для получения финальной оценки) и на практике как правило недоступна. Поэтому во время подбора гиперпараметров используются лишь `x_train, y_train`. `x_test, y_test` используются лишь для получения финальной оценки, чтобы можно было видеть разницу между разными алгоритмами подбора гиперпараметров (если она будет).

Выберите одну модель из списка: MLPClassifier, SGDClassifier, DecisionTreeClassifier, RandomForestClassifier, SVC.
Для выбранной модели произведите поиск оптимальных гиперпараметров.

**Требование**: поиск должен идти как минимум для двух гиперпараметров.

**Требование**: в конструктор моделей передавайте `random_state=1` для воспроизводимости результатов.


## 0. Обучение бейзлайн модели для проведения сравнения


In [48]:
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
# Обучите базовую модель без изменения гиперпараметров (т.е. используются гиперпараметры по умолчанию).
base_model = RandomForestClassifier(random_state=1)
base_model.fit(x_train, y_train)
y_pred = base_model.predict(x_test)
# Проанализируйте качество модели (accuracy, матрица ошибок).
print(classification_report(y_test, y_pred, zero_division=0))

              precision    recall  f1-score   support

           0       0.60      1.00      0.75         3
           1       1.00      1.00      1.00         3
           2       0.67      0.67      0.67         3
           3       0.50      0.33      0.40         3
           4       0.75      1.00      0.86         3
           5       0.00      0.00      0.00         3
           6       0.50      0.33      0.40         3
           7       0.50      1.00      0.67         3
           8       0.67      0.67      0.67         3
           9       0.50      0.33      0.40         3

    accuracy                           0.63        30
   macro avg       0.57      0.63      0.58        30
weighted avg       0.57      0.63      0.58        30



## 1. K-Fold Cross-Validation


In [49]:
# Реализуйте фунцию кросс-валидации
# Замечание: x_test, y_test не должны применятся в рамках данной функции.
from typing import Callable
from statistics import mean


def kfold_cv(
        model_fn: Callable,
        eval_fn: Callable[[np.ndarray, np.ndarray], float],
        x: np.ndarray,
        y: np.ndarray,
        n_splits: int = 5) -> float:
    """
    Parameters
    ----------
    model_fn : callable
        Функция-фабрика, что конструирует и возвращает новый объект модели.
        Например: `lambda: MLPClassifier(hidden_layer_sizes=(256,))`.
    eval_fn : callable
        Функция вида `eval_fn(labels, predictions)`, что возвращает скаляр (значение метрики).
    x : np.ndarray
        Набор признаков (размерность NxD, N - количество экземпляров, D - количество признаков).
    y : np.ndarray
        Набор меток (размерность N)
    n_splits : int, optional
        Количество фолдов (подвыборок), по умолчанию 5.
    Returns
    -------
    float
        Среднее значение метрики (что вычисляется eval_fn) по фолдам.
    """
    assert x.shape[0] == y.shape[0], 'Входные данные отличаются по колличеству'

    def get_splits(i: int) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        x_split: list = np.split(x.copy(), n_splits)
        y_split: list = np.split(y.copy(), n_splits)
        test_x = x_split.pop(i)
        test_y = y_split.pop(i)
        return (np.vstack(x_split), np.vstack(y_split).flatten(), test_x, test_y)

    metrics_vals = []
    for idx in range(n_splits):
        _x_train, _y_train, _x_test, _y_test = get_splits(idx)
        model = model_fn()
        model.fit(_x_train, _y_train)
        predictions = model.predict(_x_test)
        metrics_vals.append(eval_fn(_y_test, predictions))
    return mean(metrics_vals)

In [50]:
# Убедитесь в корректности работы функции кросс-валидации.
from sklearn.metrics import accuracy_score


kfold_cv(
    lambda: RandomForestClassifier(random_state=1),
    accuracy_score,
    x_train,
    y_train)

0.3909090909090909

## 2. Grid search


In [51]:
from typing import Iterable
from time import time
from multiprocessing import Process, Manager

In [52]:
# 1. Реализуйте алгоритм поиска гиперпараметров grid search.
# 2. Запустите поиск гиперпараметров, замерьте время работы алгоритма.
# 3. Выведите найденные значения гиперпараметров и время работы.
# Замечание: x_test, y_test не должны применятся в рамках данного алгоритма.
# Замечание: убедитесь, что гиперпараметры по умолчанию включены в пространство поиска.
# Требование: используйте kfold_cv для получения значения метрики в рамках одной итерации поиска гиперпараметров.


def log10_range(_from, _to):
    if _from == 0:
        raise Exception('Начальное значение не может быть нулём')
    while _from < _to:
        _from *= 10
        yield _from


def kfold_grid_search_worker(
        x: np.ndarray,
        y: np.ndarray,
        model_class,
        eval_fn: Callable[[np.ndarray, np.ndarray], float],
        param1: str,
        param2: str,
        param_space: dict[str, Iterable],
        rmlist: list):
    t1 = time()
    best_metric = 0.0
    best_params = {}
    for p1 in param_space[param1]:
        for p2 in param_space[param2]:
            params = {
                param1: p1,
                param2: p2
            }
            metric = kfold_cv(
                lambda: model_class(random_state=1, **params),
                eval_fn,
                x,
                y
            )
            if metric > best_metric:
                best_metric = metric
                best_params = params
    t1 = time() - t1
    print(
        f"""
        ----------------------------
        Worker(p1={param1},p1={param2})
        Execution time: {t1:.2f} 
        Best metric: {best_metric=:.2f} 
        Result: {best_params}
        ----------------------------"""
    )
    rmlist.append((best_metric, best_params))


def kfold_grid_search(
    x: np.ndarray,
    y: np.ndarray,
    model_class,
    eval_fn: Callable[[np.ndarray, np.ndarray], float],
    param_space: dict[str, Iterable],
):
    divided = []
    for key1, _ in param_space.items():
        for key2, _ in param_space.items():
            if key1 == key2:
                continue
            if (key1, key2) in divided or \
                    (key2, key1) in divided:
                continue
            divided.append((key1, key2))
    result_manager = Manager()
    rmlist = result_manager.list()
    ps = [
        Process(
            target=kfold_grid_search_worker,
            args=(x, y, model_class, eval_fn, param1, param2, param_space, rmlist,))
        for param1, param2 in divided
    ]
    t1 = time()
    [p.start() for p in ps]
    [p.join() for p in ps]
    print(f'Total processing time {time() - t1}')
    merged_params = {}
    for _, params in list(reversed(sorted(list(rmlist), key=lambda tup: tup[0]))):
        for key, value in params.items():
            cur = merged_params.get(key, None)
            if cur is None:
                merged_params[key] = value
    return merged_params


param_space = {
    'n_estimators': log10_range(1, 1000),
    'criterion': ['gini', 'entropy', 'log_loss'],
    'min_samples_leaf': log10_range(1, 10 ** 4),
    'min_weight_fraction_leaf': log10_range(10 ** -5, 0.1),
    'max_features': ['sqrt', 'log2', None],
    'min_impurity_decrease': log10_range(10 ** -5, 1),
    'max_depth': log10_range(10, 100),
    'min_samples_split': log10_range(2, 1000),
    'bootstrap': [True, False],
    'warm_start': [True, False],
    'ccp_alpha': log10_range(10 ** -5, 1),
}

found_params = kfold_grid_search(
    x_train,
    y_train,
    RandomForestClassifier,
    accuracy_score,
    param_space)


        ----------------------------
        Worker(p1=n_estimators,p1=max_depth)
        Execution time: 0.13 
        Best metric: best_metric=0.25 
        Result: {'n_estimators': 10, 'max_depth': 100}
        ----------------------------

        ----------------------------
        Worker(p1=n_estimators,p1=min_samples_split)
        Execution time: 0.31 
        Best metric: best_metric=0.16 
        Result: {'n_estimators': 10, 'min_samples_split': 20}
        ----------------------------

        ----------------------------
        Worker(p1=n_estimators,p1=min_samples_leaf)
        Execution time: 0.53 
        Best metric: best_metric=0.13 
        Result: {'n_estimators': 10, 'min_samples_leaf': 10}
        ----------------------------

        ----------------------------
        Worker(p1=n_estimators,p1=min_weight_fraction_leaf)
        Execution time: 0.95 
        Best metric: best_metric=0.25 
        Result: {'n_estimators': 10, 'min_weight_fraction_leaf': 0.0001}


In [53]:
from pprint import pprint
pprint(found_params)

{'bootstrap': False,
 'ccp_alpha': 0.0001,
 'criterion': 'entropy',
 'max_depth': 100,
 'max_features': 'sqrt',
 'min_impurity_decrease': 0.0001,
 'min_samples_leaf': 10,
 'min_samples_split': 20,
 'min_weight_fraction_leaf': 0.0001,
 'n_estimators': 1000,
 'warm_start': True}


In [54]:
# Используйте найденные гиперпараметры для обучения модели.
# Протестируйте модель на x_test, y_test.
# Сравните полученные результаты с теми, что получены в пункте 0.

from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
new_model = RandomForestClassifier(random_state=1, **found_params)
new_model.fit(x_train, y_train)

print('Base model')
y_pred = base_model.predict(x_test)
print(classification_report(y_test, y_pred, zero_division=0))

print('Found params model')
y_pred = new_model.predict(x_test)
print(classification_report(y_test, y_pred, zero_division=0))

Base model
              precision    recall  f1-score   support

           0       0.60      1.00      0.75         3
           1       1.00      1.00      1.00         3
           2       0.67      0.67      0.67         3
           3       0.50      0.33      0.40         3
           4       0.75      1.00      0.86         3
           5       0.00      0.00      0.00         3
           6       0.50      0.33      0.40         3
           7       0.50      1.00      0.67         3
           8       0.67      0.67      0.67         3
           9       0.50      0.33      0.40         3

    accuracy                           0.63        30
   macro avg       0.57      0.63      0.58        30
weighted avg       0.57      0.63      0.58        30

Found params model
              precision    recall  f1-score   support

           0       0.67      0.67      0.67         3
           1       1.00      1.00      1.00         3
           2       0.67      0.67      0.67     

## 3. Random search


In [55]:
# 1. Реализуйте алгоритм поиска гиперпараметров random search.
# 2. Запустите поиск гиперпараметров, замерьте время работы алгоритма.
# 3. Выведите найденные значения гиперпараметров и время работы.
# Замечание: x_test, y_test не должны применятся в рамках данного алгоритма.
# Замечание: убедитесь, что гиперпараметры по умолчанию включены в пространство поиска.
# Требование: используйте kfold_cv для получения значения метрики в рамках одной итерации поиска гиперпараметров.
# Требование: количество итераций должно быть меньше в сравнении с grid search.
from random import uniform, randint, getrandbits
import sys

# Максимальное колличество итераций для предыдущей модели
# ccp_alpha X min_impurity_decrease = 5 X 5 = 25
N_ITERS = 24


def kfold_random_search_worker(
        x: np.ndarray,
        y: np.ndarray,
        model_class,
        eval_fn: Callable[[np.ndarray, np.ndarray], float],
        param1: str,
        param2: str,
        param_space: dict[str, Iterable],
        rmlist: list):
    t1 = time()
    best_metric = 0.0
    best_params = {}

    params_count = 0
    for p1 in param_space[param1]:
        for p2 in param_space[param2]:
            params_count += 1

    if params_count < N_ITERS:
        for p1 in param_space[param1]:
            for p2 in param_space[param2]:
                params = {
                    param1: p1,
                    param2: p2
                }
                metric = kfold_cv(
                    lambda: model_class(random_state=1, **params),
                    eval_fn,
                    x,
                    y
                )
                if metric > best_metric:
                    best_metric = metric
                    best_params = params
    else:
        def get_typed_vlue(p0, iter):
            if isinstance(p0, str):
                return list(iter)[randint[0, len(iter) - 1]]
            if isinstance(p0, int):
                return randint(min(iter), max(iter))
            if isinstance(p0, float):
                return uniform(min(iter), max(iter))
            if isinstance(p1, bool):
                return bool(getrandbits(1))

        def gen_random_params():
            p1_iter = param_space[param1]
            p2_iter = param_space[param2]
            p01 = list(p1_iter)[0]
            p02 = list(p1_iter)[0]
            p1 = get_typed_vlue(p01, p1_iter)
            p2 = get_typed_vlue(p02, p2_iter)
            return p1, p2

        for _ in range(N_ITERS):
            p1, p2 = gen_random_params()
            params = {
                param1: p1,
                param2: p2
            }
            metric = kfold_cv(
                lambda: model_class(random_state=1, **params),
                eval_fn,
                x,
                y
            )
            if metric > best_metric:
                best_metric = metric
                best_params = params
    t1 = time() - t1
    print(
        f"""
        ----------------------------
        Worker(p1={param1},p1={param2})
        Execution time: {t1:.2f} 
        Best metric: {best_metric=:.2f} 
        Result: {best_params}
        ----------------------------"""
    )
    rmlist.append((best_metric, best_params))


def kfold_random_search(
    x: np.ndarray,
    y: np.ndarray,
    model_class,
    eval_fn: Callable[[np.ndarray, np.ndarray], float],
    param_space: dict[str, Iterable],
):
    divided = []
    for key1, _ in param_space.items():
        for key2, _ in param_space.items():
            if key1 == key2:
                continue
            if (key1, key2) in divided or \
                    (key2, key1) in divided:
                continue
            divided.append((key1, key2))
    result_manager = Manager()
    rmlist = result_manager.list()
    ps = [
        Process(
            target=kfold_random_search_worker,
            args=(x, y, model_class, eval_fn, param1, param2, param_space, rmlist,))
        for param1, param2 in divided
    ]
    t1 = time()
    [p.start() for p in ps]
    [p.join() for p in ps]
    print(f'Total processing time {time() - t1}')
    merged_params = {}
    for _, params in list(reversed(sorted(list(rmlist), key=lambda tup: tup[0]))):
        for key, value in params.items():
            cur = merged_params.get(key, None)
            if cur is None:
                merged_params[key] = value
    return merged_params


param_space = {
    'n_estimators': (1, 1000),
    'criterion': ['gini', 'entropy', 'log_loss'],
    'min_samples_leaf': (1, 10 ** 4),
    'min_weight_fraction_leaf': (10 ** -5, 0.1),
    'max_features': ['sqrt', 'log2', None],
    'min_impurity_decrease': (10 ** -5, 1),
    'max_depth': (10, 100),
    'min_samples_split': (2, 1000),
    'bootstrap': [True, False],
    'warm_start': [True, False],
    'ccp_alpha': (10 ** -5, 1),
}

found_params = kfold_random_search(
    x_train,
    y_train,
    RandomForestClassifier,
    accuracy_score,
    param_space)


        ----------------------------
        Worker(p1=min_samples_leaf,p1=warm_start)
        Execution time: 13.48 
        Best metric: best_metric=0.39 
        Result: {'min_samples_leaf': 1, 'warm_start': True}
        ----------------------------

        ----------------------------
        Worker(p1=min_impurity_decrease,p1=bootstrap)
        Execution time: 13.59 
        Best metric: best_metric=0.43 
        Result: {'min_impurity_decrease': 1e-05, 'bootstrap': False}
        ----------------------------

        ----------------------------
        Worker(p1=min_samples_split,p1=bootstrap)
        Execution time: 13.88 
        Best metric: best_metric=0.43 
        Result: {'min_samples_split': 2, 'bootstrap': False}
        ----------------------------

        ----------------------------
        Worker(p1=min_weight_fraction_leaf,p1=warm_start)
        Execution time: 14.61 
        Best metric: best_metric=0.39 
        Result: {'min_weight_fraction_leaf': 1e-05, 'wa

In [56]:
from pprint import pprint
pprint(found_params)

{'bootstrap': False,
 'ccp_alpha': 1e-05,
 'criterion': 'entropy',
 'max_depth': 100,
 'max_features': 'sqrt',
 'min_impurity_decrease': 1e-05,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 1e-05,
 'n_estimators': 1000,
 'warm_start': True}


In [57]:
# Используйте найденные гиперпараметры для обучения модели.
# Протестируйте модель на x_test, y_test (accuracy, матрица ошибок).
# Сравните полученные результаты с теми, что получены в пункте 0.
# Сравните полученные результаты с теми, что получены в пункте 2.

from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
new_rand_model = RandomForestClassifier(random_state=1, **found_params)
new_rand_model.fit(x_train, y_train)


print('Base model')
y_pred = base_model.predict(x_test)
print(classification_report(y_test, y_pred, zero_division=0))

print('Grid search model')
y_pred = new_model.predict(x_test)
print(classification_report(y_test, y_pred, zero_division=0))

print('Rand search model')
y_pred = new_rand_model.predict(x_test)
print(classification_report(y_test, y_pred, zero_division=0))

Base model
              precision    recall  f1-score   support

           0       0.60      1.00      0.75         3
           1       1.00      1.00      1.00         3
           2       0.67      0.67      0.67         3
           3       0.50      0.33      0.40         3
           4       0.75      1.00      0.86         3
           5       0.00      0.00      0.00         3
           6       0.50      0.33      0.40         3
           7       0.50      1.00      0.67         3
           8       0.67      0.67      0.67         3
           9       0.50      0.33      0.40         3

    accuracy                           0.63        30
   macro avg       0.57      0.63      0.58        30
weighted avg       0.57      0.63      0.58        30

Grid search model
              precision    recall  f1-score   support

           0       0.67      0.67      0.67         3
           1       1.00      1.00      1.00         3
           2       0.67      0.67      0.67      

## 4. Доп. задание (опционально)


### 4.1 Bayesian optimization


Примените байесовскую оптимизацию для поиска гиперпараметров.
В качестве алгоритма используйте `BayesSearchCV` из пакета `scikit-optimize`.

Сложность: почти бесплатный балл.


In [58]:
# ! pip install scikit-optimize
from skopt import BayesSearchCV

param_space = {
    'n_estimators': (100, 1000, 'log-uniform'),
    'criterion': ['gini', 'entropy', 'log_loss'],
    'min_samples_leaf': (1, 10 ** 4, 'log-uniform'),
    'min_weight_fraction_leaf': (10 ** -5, 0.1, 'log-uniform'),
    'max_features': ['sqrt', 'log2', None],
    'min_impurity_decrease': (10 ** -5, 1, 'log-uniform'),
    'max_depth': (10, 100, 'log-uniform'),
    'bootstrap': [True, False],
    'warm_start': [True, False],
    'ccp_alpha': (10 ** -5, 1, 'log-uniform'),
}

# 1. Инстанцируйте BayesSearchCV.
RandomForestClassifier(random_state=1).score
opt = BayesSearchCV(
    RandomForestClassifier(random_state=1),
    param_space,
    n_iter=N_ITERS,
    random_state=1
)

# 2. Запустите поиск гиперпараметров, замерьте время работы алгоритма.
t1 = time()
opt.fit(x_train, y_train)
t1 = time() - t1

In [59]:
# 3. Выведите найденные значения гиперпараметров и время работы.
print(f'Время работы: {t1}')
params = dict(opt.best_params_)
pprint(params)

Время работы: 91.39804530143738
{'bootstrap': True,
 'ccp_alpha': 0.05612051514265293,
 'criterion': 'gini',
 'max_depth': 11,
 'max_features': 'sqrt',
 'min_impurity_decrease': 0.00015077907479196478,
 'min_samples_leaf': 7,
 'min_weight_fraction_leaf': 0.002460522908273142,
 'n_estimators': 822,
 'warm_start': True}


In [60]:
# Используйте найденные гиперпараметры для обучения модели.
# Протестируйте модель на x_test, y_test (accuracy, матрица ошибок).
# Сравните полученные результаты с теми, что получены в пункте 0.
# Сравните полученные результаты с теми, что получены в пункте 2.

from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
opt_model = RandomForestClassifier(random_state=1, **dict(opt.best_params_))
opt_model.fit(x_train, y_train)


print('Base model')
y_pred = base_model.predict(x_test)
print(classification_report(y_test, y_pred, zero_division=0))

print('Grid search model')
y_pred = new_model.predict(x_test)
print(classification_report(y_test, y_pred, zero_division=0))

print('Rand search model')
y_pred = new_rand_model.predict(x_test)
print(classification_report(y_test, y_pred, zero_division=0))

print('skopt search model')
y_pred = opt_model.predict(x_test)
print(classification_report(y_test, y_pred, zero_division=0))

Base model
              precision    recall  f1-score   support

           0       0.60      1.00      0.75         3
           1       1.00      1.00      1.00         3
           2       0.67      0.67      0.67         3
           3       0.50      0.33      0.40         3
           4       0.75      1.00      0.86         3
           5       0.00      0.00      0.00         3
           6       0.50      0.33      0.40         3
           7       0.50      1.00      0.67         3
           8       0.67      0.67      0.67         3
           9       0.50      0.33      0.40         3

    accuracy                           0.63        30
   macro avg       0.57      0.63      0.58        30
weighted avg       0.57      0.63      0.58        30

Grid search model
              precision    recall  f1-score   support

           0       0.67      0.67      0.67         3
           1       1.00      1.00      1.00         3
           2       0.67      0.67      0.67      